## Lectura del fichero de estado de vacunación publicado por Sanidad en  
* https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm


In [1]:
# importamos pandas, numpy y datetime
# previamente ha sido necesatio instalar el motor odfpy con un pip3 install odfpy
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

Version branch cambios 18 enero

preparamos nombre del fichero csv, dentro del directorio csv. Preparamos dos ficheros:
  * `estado_vacunacion_.csv` , última versión del mismo con el acumulado de todos los informes publicados
  * `estado_vacunacion_yearmonthday.csv`, con la versión del día en cuestión, solo con la información de ese día

In [2]:
path_directorio='/Users/mharias/documents/proyectos/covid/vacunacion/csv/'
nombre_root_fichero = 'estado_vacunacion_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=2,day=19)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

Enlace al fichero en red en formato ods

In [3]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

Leemos el fichero

In [4]:
try:
    datos_row = pd.read_excel(enlace)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210223.ods


Le echamos un vistazo para asegurar que ha ido todo bien

In [5]:
datos_row

,Unnamed: 0,Dosis entregadas Pfizer (1),Dosis entregadas Moderna (1),Dosis entregadas AstraZeneca (1),Total Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Nº Personas vacunadas(pauta completada),Fecha de la última vacuna registrada (2)
0,Andalucía,568710,36300,74500,679510,626659,0.922222,216617,2021-02-22
1,Aragón,96255,6300,11700,114255,111067,0.972098,43705,2021-02-22
2,Asturias,98015,6300,9000,113315,99545,0.878480,37949,2021-02-22
3,Baleares,49500,3200,10300,63000,52801,0.838111,20560,2021-02-22
4,Canarias,111120,7000,19200,137320,122304,0.890650,49926,2021-02-22
5,Cantabria,47535,2900,5100,55535,48935,0.881156,19854,2021-02-22
6,Castilla y Leon,211485,13500,21100,246085,223034,0.906329,86452,2021-02-22
7,Castilla La Mancha,143775,9200,18000,170975,146212,0.855166,63205,2021-02-22
8,Cataluña,489680,31400,68600,589680,504270,0.855159,181835,2021-02-22
9,C. Valenciana,247060,15700,44500,307260,273978,0.891681,99045,2021-02-22


..hacemos una copia para hacerle las transformaciones necesarias:

In [6]:
datos_hoy = datos_row.copy()

Primero renombramos columnas, añadimos luego una columna con la fecha del informe `date_pub`, y finalmente añadimos fuente y origen de datos para asegurar trazabilidad de los datos:

In [7]:
datos_hoy.columns=['ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna','Dosis entregadas AstraZeneca','Dosis entregadas','Dosis administradas','% sobre entregadas','Total pauta completada','Fecha de la ultima vacuna registrada']

In [8]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace


vamos a convertir el formato de `Fecha de la ultima vacuna registrada` para que aparezca como día/mes/año sin añadir ceros de pad..de ahi el signo menos en el caso de dia y mes (%-d/%-m). La excepción se añade para la línea de Totales, que no lleva fecha asociada:

In [9]:
def convierte(x):
    campo =  x['Fecha de la ultima vacuna registrada']
    try:
        fecha = campo.strftime('%-d/%-m/%y')
        return fecha
    except:
        return campo

In [10]:
datos_hoy['Fecha de la ultima vacuna registrada'] = datos_hoy.apply(convierte,axis=1)

..y ordenamos..

In [11]:
datos_hoy

,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Fecha de la ultima vacuna registrada,date_pub,source_name,source
0,Andalucía,568710,36300,74500,679510,626659,0.922222,216617,22/2/21,23/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,Aragón,96255,6300,11700,114255,111067,0.972098,43705,22/2/21,23/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,Asturias,98015,6300,9000,113315,99545,0.878480,37949,22/2/21,23/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,Baleares,49500,3200,10300,63000,52801,0.838111,20560,22/2/21,23/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,Canarias,111120,7000,19200,137320,122304,0.890650,49926,22/2/21,23/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,Cantabria,47535,2900,5100,55535,48935,0.881156,19854,22/2/21,23/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,Castilla y Leon,211485,13500,21100,246085,223034,0.906329,86452,22/2/21,23/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,Castilla La Mancha,143775,9200,18000,170975,146212,0.855166,63205,22/2/21,23/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,Cataluña,489680,31400,68600,589680,504270,0.855159,181835,22/2/21,23/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,C. Valenciana,247060,15700,44500,307260,273978,0.891681,99045,22/2/21,23/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [12]:
datos_hoy=datos_hoy[['date_pub','ccaa','Dosis entregadas Pfizer',
                    'Dosis entregadas Moderna','Dosis entregadas AstraZeneca','Dosis entregadas',
                    'Dosis administradas','% sobre entregadas',
                    'Total pauta completada',
                    'Fecha de la ultima vacuna registrada','source_name','source']]

Hemos visto algún informe en el que se añaden espacios al nombre de Comunidad, nos aseguramos aquí de eliminarlos en caso de que vuelvan a aparecer:

In [13]:
datos_hoy['ccaa'] = datos_hoy['ccaa'].apply(lambda x : x.rstrip().lstrip())

In [14]:
datos_hoy['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

In [15]:
datos_hoy

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Fecha de la ultima vacuna registrada,source_name,source
0,23/2/21,Andalucía,568710,36300,74500,679510,626659,0.922222,216617,22/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,23/2/21,Aragón,96255,6300,11700,114255,111067,0.972098,43705,22/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,23/2/21,Asturias,98015,6300,9000,113315,99545,0.878480,37949,22/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,23/2/21,Baleares,49500,3200,10300,63000,52801,0.838111,20560,22/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,23/2/21,Canarias,111120,7000,19200,137320,122304,0.890650,49926,22/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,23/2/21,Cantabria,47535,2900,5100,55535,48935,0.881156,19854,22/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,23/2/21,Castilla y Leon,211485,13500,21100,246085,223034,0.906329,86452,22/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,23/2/21,Castilla La Mancha,143775,9200,18000,170975,146212,0.855166,63205,22/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,23/2/21,Cataluña,489680,31400,68600,589680,504270,0.855159,181835,22/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,23/2/21,C. Valenciana,247060,15700,44500,307260,273978,0.891681,99045,22/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


guardamos el fichero de hoy en el directorio csv

In [16]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora bajamos el acumulado actual para añadirle los datos de hoy, y volverlo a subir 

In [17]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [18]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,22/2/21,Navarra,46365.0,2900.0,5800.0,55065,47557,0.863652,19341.0,NaN,21/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
696,22/2/21,País Vasco,127350.0,7900.0,19600.0,154850,112681,0.727678,44558.0,NaN,21/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
697,22/2/21,Ceuta,4320.0,400.0,800.0,5520,5209,0.943659,1086.0,NaN,21/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
698,22/2/21,Melilla,3150.0,400.0,800.0,4350,4094,0.941149,1262.0,NaN,20/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


concatenamos...

In [19]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [20]:
datos_acumulados.columns

Index(['date_pub', 'ccaa', 'Dosis entregadas Pfizer',
       'Dosis entregadas Moderna', 'Dosis entregadas AstraZeneca',
       'Dosis entregadas', 'Dosis administradas', '% sobre entregadas',
       'Total pauta completada', 'Última fecha de actualización de datos',
       'Fecha de la ultima vacuna registrada', 'source_name', 'source'],
      dtype='object')

In [21]:
datos_acumulados = datos_acumulados[['date_pub', 'ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna',
                                     'Dosis entregadas AstraZeneca',
                                     'Dosis entregadas', 'Dosis administradas','% sobre entregadas',
                                     'Total pauta completada','Última fecha de actualización de datos',
                                     'Fecha de la ultima vacuna registrada','source_name', 'source']]

In [22]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,23/2/21,Navarra,46365.0,2900.0,5800.0,55065,51816,0.940997,20090.0,NaN,22/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
716,23/2/21,País Vasco,127350.0,7900.0,19600.0,154850,114568,0.739864,44902.0,NaN,22/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
717,23/2/21,Ceuta,4320.0,400.0,800.0,5520,5526,1.001087,1087.0,NaN,22/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
718,23/2/21,Melilla,3150.0,400.0,800.0,4350,4095,0.941379,1262.0,NaN,20/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [23]:
datos_acumulados['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

y volvemos a subirlo..

In [24]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)